### Import library

In [12]:
import json 
import pandas as pd 
import numpy as np 

In [19]:
df_data = pd.read_json('airlines-1558527599826.json', lines=True)

In [57]:
df_data.columns

Index(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source',
       'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'entities', 'extended_entities',
       'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang',
       'timestamp_ms', 'retweeted_status', 'extended_tweet',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'delete'],
      dtype='object')

In [58]:
#Load one twitter 
df_data.iloc[[0]].T

,0
created_at,2019-05-22 12:20:00+00:00
id,1131172858951024640.0
id_str,1131172858951024640.0
text,La ruta de easyJet entre Londres y Menorca tra...
display_text_range,"[0, 113]"
source,"<a href=""https://about.twitter.com/products/tw..."
truncated,0.0
in_reply_to_status_id,NaN
in_reply_to_status_id_str,NaN
in_reply_to_user_id,NaN


In [59]:
# load one user data from one twitter
df_user = pd.DataFrame(pd.Series(df_data['user'][0]))
df_user

,0
id,393374091
id_str,393374091
name,EP Turismo
screen_name,EPturismo
location,"Madrid, Spain"
url,http://www.europapress.es/turismo/
description,"Todo sobre política turística, hoteles, tourop..."
translator_type,none
protected,False
verified,True
